# dp100_15 リアルタイム機械学習サービスのデプロイ

"推論"とは、トレーニング済みのモデルを使用して、モデルがトレーニングされていない新しいデータのラベルを予測することを意味する。  
こうしたモデルは多くの場合、アプリケーションで個別または少数のデータ観測のための予測を即時に要求できるようにするサービスの一部としてデプロイされる。

AzureMLでは、Azure Kubernetes Service(AKS)などのコンテナー化されたプラットフォームでホストされるサービスとして.   
モデルをデプロイすることで、リアルタイムの推論ソリューションを作成できる。

## モデルをリアルタイムサービスとしてデプロイする

モデルはリアルタイムのWebサービスとして、いくつかの種類のコンピューティングターゲットにデプロイすることができる。  
これには、ローカルコンピューティング、AzureMLコンピューティングインスタンス、Azure Container Instance(ACI)、  
Azure Kubenernetes Service(AKS)クラスター、Azure関数、IoTモジュールが含まれる。

AzureMLでは、"コンテナ"をデプロイメカニズムとして使用し、モデルとコードをパッケージ化して、  
選択したコンピューティングターゲットのコンテナーにデプロイできるイメージとして使う。

> 注:テストと開発には、ローカルサービス、コンピューティングインスタンス、またはACIへデプロイが適している。  
運用環境では、アプリケーションアーキテクチャの特定のパフォーマンス、スケーラビリティ、  
およびセキュリティのニーズを満たすターゲットにデプロイする必要がある。

リアルタイムの推論サービスとしてモデルをデプロイするには、次のタスクを実行する必要がある。

### 1.トレーニング済みモデルを登録する

モデルのトレーニングが正常に行われたあとで、AzureMLワークスペースで登録する必要がある。  
そのあと、リアルタイムサービスでは必要に応じてモデルを読み込むことができる。

ローカルファイルからモデルを登録する場合は、ここに示すように**Model**オブジェクトの**register**メソッドを使用できる。

```
from azureml.core import Model

classification_model = Model.register(workspace=ws,
                       model_name='classification_model',
                       model_path='model.pkl', # local path
                       description='A classification model')
```

また、モデルのトレーニングに使用された**Run**への参照がある場合は、以下に示すように**register_model**メソッドを使用できる。

```
run.register_model( model_name='classification_model',
                    model_path='outputs/model.pkl', # run outputs path
                    description='A classification model')
```

### 2. 推論の構成を定義する

モデルは、以下のもので構成されるサービスとしてデプロイされる。

- モデルを読み込み、送信されたデータの予測を返すスクリプト
- スクリプトが実行される環境

そのため、サービスのスクリプトと環境を定義する必要がある。

#### エントリスクリプトを作成する

サービスの"エントリスクリプト"(スコアリングスクリプトと呼ばれることも)をpyファイルとして作成する。  
次の2つの関数が含まれている必要がある。

- **init()** : サービスの初期化時に呼び出し
- **run(raw_data)** : 新しいデータがサービスに送信されるときに呼び出される

通常は、**init**関数を使ってモデルレジストリからモデルを読み込み、**run**関数を使用して入力データから予測を生成する。  
次のスクリプト例は、このパターンを示している。

```
import json
import joblib
import numpy as np
from azureml.core.model import Model

# Called when the service is loaded
def init():
    global model
    # Get the path to the registered model file and load it
    model_path = Model.get_model_path('classification_model')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    # Return the predictions as any JSON serializable format
    return predictions.tolist()
```

#### 環境の作成

サービスには、エントリスクリプトを実行するPython環境が必要で、Conda構成ファイルを使用して構成できる。  
このファイルを作成する簡単な方法は、**CondaDependencies**クラスを使用し、  
(**azureml-defaults**パッケージと、**numpy**、**pandas**などの一般的に使用されるパッケージが含まれる)規定の環境を作成し、  
その他の必要なパッケージを追加してから、環境を文字列にシリアル化して保存する。

```
from azureml.core.conda_dependencies import CondaDependencies

# Add the dependencies for your model
myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")

# Save the environment config as a .yml file
env_file = 'service_files/env.yml'
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)
```

#### InferenceConfigでスクリプトと環境を組み合わせる

エントリスクリプトと環境構成ファイルを作成した後、このようにサービスの**InferenceConfig**でそれらを組み合わせることができる。

```
from azureml.core.model import InferenceConfig

classifier_inference_config = InferenceConfig(runtime= "python",
                                              source_directory = 'service_files',
                                              entry_script="score.py",
                                              conda_file="env.yml")
```

### 3.デプロイ構成を定義する

これでエントリスクリプトと環境が用意できたので、サービスがデプロイされるコンピューティングを構成する必要がある。  
AKSクラスターにデプロイする場合は、デプロイする前にクラスターとコンピューティングターゲットを作成する必要がある。

```
from azureml.core.compute import ComputeTarget, AksCompute

cluster_name = 'aks-cluster'
compute_config = AksCompute.provisioning_configuration(location='eastus')
production_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
production_cluster.wait_for_completion(show_output=True)
```

コンピューティングターゲットが作成され、デプロイ構成が定義できるようになった。  
これにより、コンテナ化されたデプロイのターゲット固有のコンピューティング使用が設定される。

```
from azureml.core.webservice import AksWebservice

classifier_deploy_config = AksWebservice.deploy_configuration(cpu_cores = 1,
                                                              memory_gb = 1)
```

ACIのデプロイを構成するコードは似ているが、ACIのコンピューティングターゲットを明示的に作成する必要がない点が異なる。  
また、**azureml.core.webservice.AciWebservice**名前空間から**deploy_configuration**クラスを使用する必要がある。  
同様に、**azureml.core.LocalWebservice**名前空間を使用して、ローカルのDockerベースのサービスを構成できる。

> 注:Azure関数にモデルをデプロイする場合は、デプロイ構成を作成する必要はない。  
その代わりに、使用する関数トリガーの種類に基づいて、モデルをパッケージ化する必要がある。  
詳細URL : https://aka.ms/AA70rrn

### 4.モデルをデプロイする

すべての構成を準備したあとでモデルをデプロイできる。  
これを行う最も簡単な方法は、以下のように**Model**クラスの**deploy**メソッドを呼び出すこと。

```
from azureml.core.model import Model

model = ws.models['classification_model']
service = Model.deploy(workspace=ws,
                       name = 'classifier-service',
                       models = [model],
                       inference_config = classifier_inference_config,
                       deployment_config = classifier_deploy_config,
                       deployment_target = production_cluster)
service.wait_for_deployment(show_output = True)
```

ACIまたはローカルサービスの場合は、**deployment_target**パラメータを省略(または**None**)できる。

AzureMLを使用したモデルのデプロイの詳細 : https://aka.ms/AA70zfv

## リアルタイムの推論サービスを使用する

リアルタイムサービスをデプロイした後、それをクライアントアプリケーションから使用して、新しいデータケースのラベルを予測することができる。

### AzureML SDKを使用する

テストでは、AzureML SDKを使用して、デプロイされたサービスを参照する**WebService**オブジェクトの**run**メソッドを介して、  
Webサービスを呼び出すことができる。通常は、次の構造でJSON形式の**run**メソッドにデータを送信する。

```
{
  "data":[
      [0.1,2.3,4.1,2.0], // 1st case
      [0.2,1.8,3.9,2.1],  // 2nd case,
      ...
  ]
}
```

**run**メソッドからの応答は、データで送信された各ケースの予測を含むJSONコレクション。  
次のコードサンプルでは、サービスを呼び出して応答を表示する。

```
import json

# An array of new data cases
x_new = [[0.1,2.3,4.1,2.0],
         [0.2,1.8,3.9,2.1]]

# Convert the array to a serializable list in a JSON document
json_data = json.dumps({"data": x_new})

# Call the web service, passing the input data
response = service.run(input_data = json_data)

# Get the predictions
predictions = json.loads(response)

# Print the predicted class for each case.
for i in range(len(x_new)):
    print (x_new[i], predictions[i])
```

### RESTエンドポイントを使用する

運用環境では、ほとんどのクライアントアプリケーションにAzureML SDKは含まれず、RESTインターフェイスを介してサービスが使用される。  
デプロイされたサービスのエンドポイントは、AzureMLスタジオで、または次のようにSDKで**Webservice**オブジェクトの**scoring_uri**プロパティを取得することで確認できる。

```
endpoint = service.scoring_uri
print(endpoint)
```

エンドポイントがわかっている場合は、JSONデータと共にHTTP POST要求を使用してサービスを呼び出すことができる。  
次の例は、Pythonを使用してこれを行う方法を示している。

```
import requests
import json

# An array of new data cases
x_new = [[0.1,2.3,4.1,2.0],
         [0.2,1.8,3.9,2.1]]

# Convert the array to a serializable list in a JSON document
json_data = json.dumps({"data": x_new})

# Set the content type in the request headers
request_headers = { 'Content-Type':'application/json' }

# Call the service
response = requests.post(url = endpoint,
                         data = json_data,
                         headers = request_headers)

# Get the predictions from the JSON response
predictions = json.loads(response.json())

# Print the predicted class for each case.
for i in range(len(x_new)):
    print (x_new[i]), predictions[i] )
```

### 認証

運用環境では、認証の適用によるサービスへのアクセスの制限が必要になる場合がある。  
使用できる認証には、次の2種類がある。

- **Key** : 要求は、サービスに関連付けられているキーを指定することによって認証される
- **Token** : 要求は、JSON Web Token(JWT)を提供することによって認証される

規定では、認証はACIサービスに対して無効になっており、AKSサービスのキーベース設定されている(プライマリおよびセカンダリキーは自動的に生成)。  
必要に応じて、トークンベース認証を使用するようにAKSサービスを構成することができる(ACIサービスはサポートされていない)。

ワークスペースで認証されたセッションが確立されていると仮定した場合、  
サービスに関連付けられている**WebService**オブジェクトの**get_keys**メソッドを使用して、サービスのキーを取得できる。

```
primary_key, secondary_key = service.get_keys()
```

トークンベース認証の場合、クライアントアプリケーションでサービスプリンシパル認証を使用して、  
Azure Active Directory(Azure AD)を介してそのIDを検証し、サービスの**get_token**メソッドを呼び出して、期間限定のトークンを取得する必要がある。

サービスのRESTエンドポイントへの認証された呼び出しを行うには、次のように要求ヘッダーにキーまたはトークンを含める必要がある。

```
import requests
import json

# An array of new data cases
x_new = [[0.1,2.3,4.1,2.0],
         [0.2,1.8,3.9,2.1]]

# Convert the array to a serializable list in a JSON document
json_data = json.dumps({"data": x_new})

# Set the content type in the request headers
request_headers = { "Content-Type":"application/json",
                    "Authorization":"Bearer " + key_or_token }

# Call the service
response = requests.post(url = endpoint,
                         data = json_data,
                         headers = request_headers)

# Get the predictions from the JSON response
predictions = json.loads(response.json())

# Print the predicted class for each case.
for i in range(len(x_new)):
    print (x_new[i]), predictions[i] )
```

## サービスデプロイのトラブルシューティングを行う

リアルタイムサービスのデプロイには、トレーニング済みモデル、ランタイム環境の構成、スコアリングスクリプト、コンテナイメージ、  
コンテナホストなど、多数の要素がある。

失敗したデプロイや、デプロイされたサービスの使用時のエラーのトラブルシューティングは複雑な場合がある。

### サービスの状態を確認する

最初のトラブルシューティング手順として、サービス状態を確認することができる。  
そのためには、以下のようにして**状態**を調べる。

```
from azureml.core.webservice import AksWebservice

# デプロイされたサービスを取得
service = AksWebservice(name='classifier-service', workspace=ws)

# ステータス確認
print(service.state)
```

> 注:サービスの**状態**を表示するには、汎用の**WebService**オブジェクトではなく、  
コンピューティング固有のサービスの種類(たとえば**AksWebservice)を使用する必要あり

### サービスログを確認する

サービスに異常やエラーが発生した場合は、そのログを確認できる。

```
print(service.get_logs())
```

ログには、サービスのプロビジョニングと、処理された要求に関する詳細情報が含まれている。  
多くの場合、これらにより予期しないエラーの原因について洞察を得ることができる。

### ローカルコンテナにデプロイ

デプロイおよびランタイムエラーは、以下のようにローカルのDockerインスタンスにコンテナとしてサービスをデプロイすることで診断しやすくなる。

```
from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration(port=8890)
service = Model.deploy(ws, 'test-svc', [model], inference_config, deployment_config)
```

その後、SDKを使用してローカルにデプロイされたサービスをテストできる。

```
print(service.run(input_data = json_data))
```

その後、推論構成で参照されるスコアリングファイルに変更を加え、サービスを再デプロイせずに再読込することで、  
ランタイムの問題をトラブルシューティングできる(ローカルサービスでのみ行うことができる)。

```
service.reload()
print(service.run(input_data = json_data))
```

## 演習 リアルタイムサービスとしてモデルをデプロイする

### ワークスペースの接続

In [1]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.28.0 to work with 20210613


### モデルの訓練および登録

In [2]:
from azureml.core import Experiment
from azureml.core import Model
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace=ws, name="mslearn-train-diabetes")
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('data/diabetes.csv')

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Save the trained model
model_file = 'diabetes_model.pkl'
joblib.dump(value=model, filename=model_file)
run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)

# Complete the run
run.complete()

# Register the model
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

print('Model trained and registered.')

Starting experiment: mslearn-train-diabetes
Loading Data...
Training a decision tree model
Accuracy: 0.8906666666666667
AUC: 0.8778496442609085
Model trained and registered.


### モデルをWebサービスとしてデプロイする

まず、ワークスペースにどのようなモデルが登録されているかを確認する。

In [3]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

diabetes_model version: 10
	 Training context : Inline Training
	 AUC : 0.8778496442609085
	 Accuracy : 0.8906666666666667


diabetes_model version: 9
	 Training context : Pipeline
	 AUC : 0.8854221505732166
	 Accuracy : 0.9004444444444445


diabetes_model version: 8
	 Training context : Compute cluster
	 AUC : 0.8840918562273435
	 Accuracy : 0.8991111111111111


diabetes_model version: 7
	 Training context : File dataset
	 AUC : 0.8568743524381947
	 Accuracy : 0.7891111111111111


diabetes_model version: 6
	 Training context : Tabular dataset
	 AUC : 0.8568509052814499
	 Accuracy : 0.7891111111111111


diabetes_model version: 5
	 Training context : Tabular dataset
	 AUC : 0.8568509052814499
	 Accuracy : 0.7891111111111111


diabetes_model version: 4
	 Training context : Tabular dataset
	 AUC : 0.8568509052814499
	 Accuracy : 0.7891111111111111


diabetes_model version: 3
	 Training context : Tabular dataset
	 AUC : 0.8568509052814499
	 Accuracy : 0.7891111111111111


diabetes_model ve

デプロイしたいモデルを取得する。デフォルトではモデル名を指定すると最新バージョンが指定される。

In [4]:
model = ws.models['diabetes_model']
print(model.name, 'version', model.version)

diabetes_model version 10


このモデルをホストするためのWebサービスを作成するが、そのためにはいくつかコードと設定ファイルが必要になる。

In [5]:
import os

folder_name = 'diabetes_service'

# Create a folder for the web service files
experiment_folder = './' + folder_name
os.makedirs(experiment_folder, exist_ok=True)

print(folder_name, 'folder created.')

# Set path for scoring script
script_file = os.path.join(experiment_folder,"score_diabetes.py")

diabetes_service folder created.


モデルをデプロイするウェブサービスは、入力データを読み込み、ワークスペースからモデルを取得し、  
予測値を生成して返すためのPythonコードが必要になる。  
このコードをエントリースクリプト(またはスコアリングスクリプト)に保存し、Webサービスにデプロイする。

In [6]:
%%writefile $script_file
import json
import joblib
import numpy as np
from azureml.core.model import Model

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = Model.get_model_path('diabetes_model')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['not-diabetic', 'diabetic']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes)

Writing ./diabetes_service/score_diabetes.py


Webサービスはコンテナでホストされ、コンテナは初期化時に必要なPythonの依存関係をインストールする必要がある。  
この場合、スコアリングコードにはscikit-learnがひつようなので、ymlファイルを作成して、  
コンテナのホストにscikit-learnを環境にインストールするようにする。

In [7]:
from azureml.core.conda_dependencies import CondaDependencies 

# Add the dependencies for our model (AzureML defaults is already included)
myenv = CondaDependencies()
myenv.add_conda_package('scikit-learn')

# Save the environment config as a .yml file
 = os.path.join(experiment_folder,"diabetes_env.yml")
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

# Print the .yml file
with open(env_file,"r") as f:
    print(f.read())

Saved dependency info in ./diabetes_service/diabetes_env.yml
# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults

- scikit-learn
channels:
- anaconda
- conda-forge



デプロイの手順は以下の通り。

1. モデルの読み込みと使用に必要なスコアリングと環境ファイルを含む推論構成を定義
2. サービスがホストされる実行環境を定義するデプロイメント構成を定義  
ここではAzure Container Instanceを使用
3. モデルをWebサービスとしてデプロイ
4. デプロイされたサービスの状態を確認

※詳細URL : https://docs.microsoft.com/azure/machine-learning/how-to-deploy-and-where

デプロイでは、まずコンテナイメージを作成するプロセスが実行され、次にそのイメージに基づいてWebサービスを作成するプロセスが実行されるため時間がかかる。  
デプロイが正常に完了すると、ステータスが**Healthy**と表示される。

In [16]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",
                                   entry_script=script_file,
                                   conda_file=env_file)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

service_name = "diabetes-service"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-06-15 06:32:07+00:00 Creating Container Registry if not exists.
2021-06-15 06:32:07+00:00 Registering the environment.
2021-06-15 06:32:09+00:00 Building image..
2021-06-15 06:39:10+00:00 Generating deployment configuration.
2021-06-15 06:39:11+00:00 Submitting deployment to compute..
2021-06-15 06:39:50+00:00 Checking the status of deployment diabetes-service..
2021-06-15 06:41:37+00:00 Checking the status of inference endpoint diabetes-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


うまくいけばデプロイは成功して**Healthy**が表示される。  
そうでない場合は、以下のコードを使用してサービスログを取得し、トラブルシューティングに役立てることができる。

In [18]:
print(service.get_logs())

# うまく行かずに再デプロイする必要があれば、以下のコードでサービスを削除する必要がある
#service.delete()

2021-06-15T06:41:20,209334800+00:00 - rsyslog/run 
2021-06-15T06:41:20,209813000+00:00 - gunicorn/run 
2021-06-15T06:41:20,213386100+00:00 - iot-server/run 
2021-06-15T06:41:20,283973600+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-06-15T06:41:20,770021300+00:00 - iot-server/finish 1 0
2021-06-15T06:41:20,775887000+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (72)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 100
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2021-06-15 06:41:23,265 | root | INFO | Starting up app insights client
2021-06-15 06:41:23,266 | root | INFO | Starting up request id generator
2021-06-15 06:41:23,266 | root | INFO | Starting up app insight hooks
2021-06-15 06:41:23,267 | root | INFO | Invoking user's init function
2021-06-15 06:41:24,008 | root | INFO | Users's init has completed su

以下のコードを実行することで、ワークスペース内のWebサービスの名前を取得することができる。

In [19]:
for webservice_name in ws.webservices:
    print(webservice_name)

diabetes-service


### Webサービスを使う

サービスがデプロイされると、今度はクライアントアプリケーションからサービスを使用できるようになる。

In [20]:
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22]]
print ('Patient: {}'.format(x_new[0]))

# リストをjsonファイルのシリアライズ可能なリストへ変換
input_json = json.dumps({"data": x_new})

# 入力データを渡してWebサービスを呼び出す(Webサービスはバイナリ形式のデータも受け付ける)
predictions = service.run(input_data = input_json)

# 予測されるクラスを取得する　※これが最初の(かつ唯一の)クラスになる
predicted_classes = json.loads(predictions)
print(predicted_classes[0])

Patient: [2, 180, 74, 24, 21, 23.9091702, 1.488172308, 22]
diabetic


また、複数のデータを送信すると、それぞれの予測値を得ることができる。

In [22]:
import json

# This time our input is an array of two feature arrays
x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array or arrays to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data
predictions = service.run(input_data = input_json)

# Get the predicted classes.
predicted_classes = json.loads(predictions)
   
for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

Patient [2, 180, 74, 24, 21, 23.9091702, 1.488172308, 22] diabetic
Patient [0, 148, 58, 11, 179, 39.19207553, 0.160829008, 45] not-diabetic


上記のコードではAzureML　SDKを使用してコンテナ化されたWebサービスに接続し、それを使用して予測値を生成している。  
本番環境では、AzureML SDKを使用せず、単にWebサービスへのHTTPリクエストを行うビジネスアプリケーションによってモデルが消費される可能性がある。

これらのアプリケーションがリクエストを送信しなければならないURLを決めよう。

In [23]:
endpoint = service.scoring_uri
print(endpoint)

http://37bbfafd-1a05-437d-9cdf-3ece0ba3e5c1.westus2.azurecontainer.io/score


これでエンドポイントのURLがわかったので、アプリケーションは単純にHTTPリクエストを行い、jsonフォーマットのデータを送信し、  
予測されたクラスを受信することができる。

In [24]:
import requests
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Set the content type
headers = { 'Content-Type':'application/json' }

predictions = requests.post(endpoint, input_json, headers = headers)
predicted_classes = json.loads(predictions.json())

for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

Patient [2, 180, 74, 24, 21, 23.9091702, 1.488172308, 22] diabetic
Patient [0, 148, 58, 11, 179, 39.19207553, 0.160829008, 45] not-diabetic


Webサービスを、認証を必要としないAzure Containter Instance(ACI)サービスとして展開した。  
これは開発やテストには良いが、本番環境ではAzure Kubernetes Service(AKS)クラスタにデプロイし、  
トークンベースの認証を有効にすることを検討する必要がある。
この場合、RESTリクエストに**Authorization**ヘッダを含める必要がある。

### サービスの削除

サービスが不要になったら、不要な料金が発生しないように削除しておく。

In [25]:
service.delete()
print ('Service deleted.')

Service deleted.


モデルをサービスとして公開することについての詳細情報 : https://docs.microsoft.com/azure/machine-learning/how-to-deploy-and-where

## 知識チェック

1. Azure Machine Learning 用の Python SDK を使用して、モデルをトレーニングしました。  
高いスケーラビリティとセキュリティを備え、コンテナー化されたリアルタイム サービスとしてモデルをデプロイする必要があります。  
このサービスをホストするには、どのような種類のコンピューティングを作成する必要がありますか?
    - Azure Kubernetes Service (AKS) 推論クラスター
    - GPU を使用するコンピューティング インスタンス
    - 複数のノードがあるトレーニング クラスター。


2. モデルをリアルタイムの推論サービスとしてデプロイしています。  
サービスのエントリ スクリプトには、どのような関数が含まれている必要がありますか?
    - main() と score()
    - base() と train()
    - init() と run()

↓解答

1. Azure Kubernetes Service (AKS) 推論クラスター
    - モデルをスケーラブルで、セキュリティで保護され、コンテナー化されたサービスとしてデプロイするには、  
    AKS クラスターを使用する必要があります。


2. init() と run()
    - エントリ (スコアリング) スクリプトでは、init 関数と run 関数を実装する必要があります。